# 0. Import

In [32]:
# Data Wrangling
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

# EDA
import klib

import gc
import warnings
warnings.filterwarnings("ignore")

# 1. Load Data

In [132]:
user_cols = ['신청서번호','유저번호','유저생년월일','유저성별','생성일시','한도조회당시유저신용점수','연소득','근로형태','입사연월',
             '고용형태','주거소유형태','대출희망금액','대출목적','개인회생자여부','개인회생자납입완료여부','기대출수','기대출금액']

loan_cols = ['신청서번호','한도조회일시','금융사번호','상품번호','승인한도','승인금리','신청여부']

log_cols = ['유저번호','행동명','행동일시','운영체재','앱버전','한도조회월일']

In [133]:
user = pd.read_csv("data/user_spec.csv", encoding='UTF-8', header=0, names=user_cols)
loan = pd.read_csv("data/loan_result.csv", encoding='UTF-8', header=0, names=loan_cols)
log = pd.read_csv("data/log_data.csv", encoding='UTF-8', header=0, names=log_cols)

### -User data

In [11]:
user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1394216 entries, 0 to 1394215
Data columns (total 17 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   신청서번호         1394216 non-null  int64  
 1   유저번호          1394216 non-null  int64  
 2   유저생년월일        1381255 non-null  float64
 3   유저성별          1381255 non-null  float64
 4   생성일시          1394216 non-null  object 
 5   한도조회당시유저신용점수  1289101 non-null  float64
 6   연소득           1394126 non-null  float64
 7   근로형태          1394131 non-null  object 
 8   입사연월          1222456 non-null  float64
 9   고용형태          1394131 non-null  object 
 10  주거소유형태        1394131 non-null  object 
 11  대출희망금액        1394131 non-null  float64
 12  대출목적          1394131 non-null  object 
 13  개인회생자여부       806755 non-null   float64
 14  개인회생자납입완료여부   190862 non-null   float64
 15  기대출수          1195660 non-null  float64
 16  기대출금액         1080442 non-null  float64
dtypes: float64(10), int64(2), o

### -Loan data

In [12]:
loan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13527363 entries, 0 to 13527362
Data columns (total 7 columns):
 #   Column  Dtype  
---  ------  -----  
 0   신청서번호   int64  
 1   한도조회일시  object 
 2   금융사번호   int64  
 3   상품번호    int64  
 4   승인한도    float64
 5   승인금리    float64
 6   신청여부    float64
dtypes: float64(3), int64(3), object(1)
memory usage: 722.4+ MB


### -Log data

In [13]:
log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17843993 entries, 0 to 17843992
Data columns (total 6 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   유저번호    int64 
 1   행동명     object
 2   행동일시    object
 3   운영체재    object
 4   앱버전     object
 5   일코드     object
dtypes: int64(1), object(5)
memory usage: 816.8+ MB


## 2. Data Preprocessing

### -User data

In [34]:
#결측치 확인
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
#            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return "완료"

display(check_missing_col(user))

결측치가 있는 컬럼은: 유저생년월일입니다
해당 컬럼에 총 12961개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 유저성별입니다
해당 컬럼에 총 12961개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 한도조회당시유저신용점수입니다
해당 컬럼에 총 105115개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 연소득입니다
해당 컬럼에 총 90개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 근로형태입니다
해당 컬럼에 총 85개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 입사연월입니다
해당 컬럼에 총 171760개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 고용형태입니다
해당 컬럼에 총 85개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 주거소유형태입니다
해당 컬럼에 총 85개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 대출희망금액입니다
해당 컬럼에 총 85개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 대출목적입니다
해당 컬럼에 총 85개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 개인회생자여부입니다
해당 컬럼에 총 587461개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 개인회생자납입완료여부입니다
해당 컬럼에 총 1203354개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 기대출수입니다
해당 컬럼에 총 198556개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 기대출금액입니다
해당 컬럼에 총 313774개의 결측치가 존재합니다.


'완료'

### User_spec
 - application_id(신청서번호) = 모두 고유값, 결측치 x
 - user_id(유저번호) = 중복값 존재, 결측치 x
 - <span style="color:red">birth_year(유저생년월일) = 12961개의 결측치가 존재, 어린나이인 이용자들도 존재함(2008년생까지)</span>
 - <span style="color:red">gender(유저성별) = 12961개의 결측치가 존재(0:여자, 1:남자)</span>
 - insert_time(생성일시) = 중복값 존재, 결측치 x
 - <span style="color:red">credit_score(신용점수) = 105115개의 결측치가 존재</span>
 - <span style="color:red">yearly_income(연소득) = 90개의 결측치가 존재</span>
 - <span style="color:red">income_type(근로형태) = 85개의 결측치가 존재</span>
 - <span style="color:red">company_enter_month(입사연월) = 171760개의 결측치가 존재</span>
 - <span style="color:red">employment_type(고용형태) = 85개의 결측치가 존재</span>
 - <span style="color:red">houseown_type(주거소유형태) = 85개의 결측치가 존재</span>
 - <span style="color:red">desired_amount(대출희망금액) = 85개의 결측치가 존재</span>
 - <span style="color:red">purpose(대출목적) = 85개의 결측치가 존재</span>
 - <span style="color:red">personal_rehabilitation_yn(개인회생자여부) = 587461개의 결측치가 존재</span>
 - <span style="color:red">personal_rehabilitation_complete_yn(개인회생자납입완료여부) = 1203354개의 결측치가 존재</span>
 - <span style="color:red">existing_loan_cnt(기대출수) = 198556개의 결측치가 존재</span>
 - <span style="color:red">existing_loan_amt(기대출금액) = 313774개의 결측치가 존재</span>

### Data leakage 문제없는 열 처리

In [35]:
#유저생년월일 이상치 처리(미성년자) > 개인정보 기입후 대출조회까지는 대부분 하지 않음(그냥 앱이용 정도만 해본걸로 판단됨)

#유저생년월일 결측치 처리(존재값 처리)
user['유저생년월일'] = user['유저생년월일'].fillna(user.groupby('유저번호')['유저생년월일'].transform('mean'))

#유저생년월일 결측치 처리(단일결측값 처리)
user['유저생년월일'] = user['유저생년월일'].fillna('기입안함')

#이용자의 단순 행동적인 요인이라고 판단되어 위 사항으로 대체함

#유저생년월일 데이터 타입 변경(float64 > str)
user['유저생년월일'] = user['유저생년월일'].apply(lambda x : str(x)[:4])

In [36]:
#성별 이상치 처리(없음)

#성별 결측치 처리(존재값 처리)
user['유저성별'] = user['유저성별'].fillna(user.groupby('유저번호')['유저성별'].transform('mean'))

#성별 결측치 처리(단일결측값 처리)
user['유저성별'] = user['유저성별'].fillna('기입안함')
#이용자의 단순 행동적인 요인이라고 판단되어 위 사항으로 대체함

#유저성별 데이터 값 변경
user.loc[user['유저성별'] == 0.0, '유저성별'] = '여자'
user.loc[user['유저성별'] == 1.0, '유저성별'] = '남자'

In [37]:
#한도조회당시유저신용점수 이상치 처리(없음)

#한도조회당시유저신용점수 결측치 처리(신용점수 조회당시 조회자의 신용거래 정보부족으로 생각됨)
#결측치인 유저들을 보면 대체로 모두 결측치이거나 스펙서가 하나밖에 존재하지 않음
#한도조회당시유저신용점수가 결측치인 신청서번호들이 대체로 추천된 상품을 신청하지 않는 것으로 판단되어 신용점수의 값의 의미를 살리고자
#결측치를 0으로 대체함
user['한도조회당시유저신용점수'] = user['한도조회당시유저신용점수'].fillna(0.0)

In [38]:
#근로형태 결측치 처리(추천상품에 대한 만족도가 떨어져서 입력하지 않은 것으로 단순 행동적 요인이라고 판단됨)
user['근로형태'] = user['근로형태'].fillna('기입안함')

In [43]:
#입사연월 결측치 처리(대부분 근로형태가 기타소득인것으로 보아 스펙서 생성당시 회사에 입사하지 않은 것으로 판단됨)
user['입사연월'] = user['입사연월'].fillna('기입안함.0')

#입사연월 데이터 형태 변경(float64 > object)
user['입사연월'] = user['입사연월'].apply(lambda x: str(x).split(".")[0])

#입사연월 이상치 처리(생년월일보다 입사연월이 빠른경우)
#유저생년월일과 비교하여 처리
index = user.query('입사연월!="기입안함" & 유저생년월일!="기입안함"')[user.query('입사연월!="기입안함" & 유저생년월일!="기입안함"')['입사연월'].apply(lambda x : int(x[:4])) < user.query('입사연월!="기입안함" & 유저생년월일!="기입안함"')['유저생년월일'].apply(lambda x : int(x))]
user.loc[index.index,'입사연월'] = "기입이상"

In [44]:
#고용형태 이상치 처리(없음)

#고용형태 결측치 처리(추천상품에 대한 만족도가 떨어져서 입력하지 않은 것으로 단순 행동적 요인이라고 판단됨)
user['고용형태'] = user['고용형태'].fillna('기입안함')

In [45]:
#주거소유형태 이상치 처리(없음)

#주거소유형태 결측치 처리(추천상품에 대한 만족도가 떨어져서 입력하지 않은 것으로 단순 행동적 요인이라고 판단됨)
user['주거소유형태'] = user['주거소유형태'].fillna('기입안함')

In [46]:
#대출목적 이상치 처리(없음)

#대출목적 결측치 처리(추천상품에 대한 만족도가 떨어져서 입력하지 않은 것으로 단순 행동적 요인이라고 판단됨)
user['대출목적'] = user['대출목적'].fillna('기입안함')

In [47]:
#개인회생자 = 채무를 더이상 갚을 능력이 안된다고 법원이 판단하여 채무를 탕감 또는 면책해준 자

#개인회생자여부 이상치 처리(없음)

#개인회생자여부 결측치 처리(개인회생자여부가 결측치면 개인회생자납입완료여부도 결측치임)
user['개인회생자여부'] = user['개인회생자여부'].fillna('누락')

#개인회생자여부 데이터 값 변경
user.loc[user['개인회생자여부'] == 0.0, '개인회생자여부'] = '비개인회생자'
user.loc[user['개인회생자여부'] == 1.0, '개인회생자여부'] = '개인회생자'

In [48]:
#개인회생자납입완료여부 이상치 처리(없음)

#개인회생자납입완료여부 결측치 처리(개인회생자여부와 마찬가지로 단순 누락이라고 판단함)
user['개인회생자납입완료여부'] = user['개인회생자납입완료여부'].fillna('누락')

#개인회생자납입완료여부 데이터 값 변경
user.loc[user['개인회생자납입완료여부'] == 0.0, '개인회생자납입완료여부'] = '납입중'
user.loc[user['개인회생자납입완료여부'] == 1.0, '개인회생자납입완료여부'] = '납입완료'

### Data leakage 문제있는 열 처리

In [49]:
#생성일시 기준 3,4,5월 데이터는 train 6월 데이터는 test로 데이터 split(data leakage 방지)
user['생성일시'] = user['생성일시'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
user = user.sort_values(by="생성일시")

user_train = user[(user['생성일시'].dt.month)<6].reset_index(drop=True)
user_test = user[(user['생성일시'].dt.month)==6].reset_index(drop=True)

display(user_train.head(3))
display(user_test.head(3))

,신청서번호,유저번호,유저생년월일,유저성별,생성일시,한도조회당시유저신용점수,연소득,근로형태,입사연월,고용형태,주거소유형태,대출희망금액,대출목적,개인회생자여부,개인회생자납입완료여부,기대출수,기대출금액
0,566528,681184,1974,남자,2022-03-01 00:11:35,580.0,8000000.0,OTHERINCOME,기입안함,기타,전월세,1000000.0,생활비,누락,누락,4.0,20000000.0
1,180433,623737,1997,남자,2022-03-01 00:12:05,740.0,12000000.0,FREELANCER,202202,일용직,기타가족소유,3000000.0,생활비,누락,누락,3.0,11000000.0
2,1657888,752985,1997,남자,2022-03-01 00:12:06,580.0,12000000.0,FREELANCER,202102,기타,기타가족소유,3000000.0,생활비,누락,누락,7.0,33000000.0


,신청서번호,유저번호,유저생년월일,유저성별,생성일시,한도조회당시유저신용점수,연소득,근로형태,입사연월,고용형태,주거소유형태,대출희망금액,대출목적,개인회생자여부,개인회생자납입완료여부,기대출수,기대출금액
0,501277,542827,1980,남자,2022-06-01 00:00:57,660.0,65000000.0,EARNEDINCOME,202001,계약직,자가,65000000.0,대환대출,비개인회생자,누락,5.0,151000000.0
1,731545,8641,1979,남자,2022-06-01 00:12:08,610.0,51000000.0,PRACTITIONER,201601,정규직,전월세,3000000.0,생활비,비개인회생자,누락,7.0,60000000.0
2,1343197,710829,1993,남자,2022-06-01 00:12:22,600.0,30000000.0,OTHERINCOME,기입안함,기타,자가,5000000.0,생활비,비개인회생자,누락,8.0,66000000.0


In [50]:
#연소득 이상치 처리(없음)

#연소득 결측치 처리(추천상품에 대한 만족도가 떨어져서 입력하지 않은 것으로 단순 행동적 요인이라고 판단됨)
#존재값 처리
user_train['연소득'] = user_train['연소득'].fillna(user_train.groupby('유저번호')['연소득'].transform('mean'))
user_test['연소득'] = user_test['연소득'].fillna(user_train.groupby('유저번호')['연소득'].transform('mean'))

#단일결측값 처리
user_train['연소득'] = user_train['연소득'].fillna(user_train['연소득'].mean())
user_test['연소득'] = user_test['연소득'].fillna(user_train['연소득'].mean())

#데이터 통일성을 위해 십만단위에서 반올림
user_train['연소득'] = user_train['연소득'].apply(lambda x : round(x,-5))
user_test['연소득'] = user_test['연소득'].apply(lambda x : round(x,-5))

In [51]:
#대출희망금액 이상치 처리(희망금액이 0원인 경우) > 대부분 대출목적이 생활비인 것으로 보아 정확한 필요 금액이 없고 금리나 한도를 보고 임의적으로 대출하는 것으로 보임
#다출희망금액이 0원인 것은 그대로 살려두어 나중에 군집분석에 활용하는게 좋아보임
#대출희망금액이 너무 큰것도 마찬가지

#대출희망금액 결측치 처리(추천상품에 대한 만족도가 떨어져서 입력하지 않은 것으로 단순 행동적 요인이라고 판단됨)
#존재값 처리
user_train['대출희망금액'] = user_train['대출희망금액'].fillna(user_train.groupby('유저번호')['대출희망금액'].transform('mean'))
user_test['대출희망금액'] = user_test['대출희망금액'].fillna(user_train.groupby('유저번호')['대출희망금액'].transform('mean'))

#단일결측값 처리
user_train['대출희망금액'] = user_train['대출희망금액'].fillna(user_train['대출희망금액'].mean())
user_test['대출희망금액'] = user_test['대출희망금액'].fillna(user_train['대출희망금액'].mean())

#데이터 통일성을 위해 십만단위에서 반올림
user_train['대출희망금액'] = user_train['대출희망금액'].apply(lambda x : round(x,-5))
user_test['대출희망금액'] = user_test['대출희망금액'].apply(lambda x : round(x,-5))

In [52]:
#기대출수 이상치 처리(기대출수가 너무 많은 경우) > 실제로 가능한 정도의 값이라고 판단됨(대출목적이 사업자금인 것으로 보아 자주 대출상품조회를 한것으로 판단됨)

#기대출수 결측치 처리(단순 누락이거나 신규유저로 판단함)
#존재값 처리
user_train['기대출수'] = user_train['기대출수'].fillna(user_train.groupby('유저번호')['기대출수'].transform('mean'))
user_test['기대출수'] = user_test['기대출수'].fillna(user_train.groupby('유저번호')['기대출수'].transform('mean'))

#단일결측값 처리
user_train['기대출수'] = user_train['기대출수'].fillna(0.0)
user_test['기대출수'] = user_test['기대출수'].fillna(0.0)

#데이터 통일성을 위해 소수점첫째자리에서 반올림
user_train['기대출수'] = user_train['기대출수'].apply(lambda x : round(x,1))
user_test['기대출수'] = user_test['기대출수'].apply(lambda x : round(x,1))

In [53]:
#게대출금액 이상치 처리(기대출금액이 너무 많은 경우) > 실제로 가능한 정도의 값이라고 판단됨(대출목적이 사업자금인 것으로 보아 자주 대출상품조회를 한것으로 판단됨)

#기대출금액 결측치 처리(단순 누락이거나 신규유저로 판단함)
#존재값 처리
user_train['기대출금액'] = user_train['기대출금액'].fillna(user_train.groupby('유저번호')['기대출금액'].transform('mean'))
user_test['기대출금액'] = user_test['기대출금액'].fillna(user_train.groupby('유저번호')['기대출금액'].transform('mean'))

#단일결측값 처리
user_train['기대출금액'] = user_train['기대출금액'].fillna(0.0)
user_test['기대출금액'] = user_test['기대출금액'].fillna(0.0)

#데이터 통일성을 위해 십만단위에서 반올림
user_train['기대출금액'] = user_train['기대출금액'].apply(lambda x : round(x,-5))
user_test['기대출금액'] = user_test['기대출금액'].apply(lambda x : round(x,-5))

In [54]:
#결측치 확인
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
#            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return "완료"

display(check_missing_col(user_train))
display(check_missing_col(user_test))

결측치가 존재하지 않습니다


'완료'

결측치가 존재하지 않습니다


'완료'

In [55]:
display(user_train.head(3))
display(user_test.head(3))

,신청서번호,유저번호,유저생년월일,유저성별,생성일시,한도조회당시유저신용점수,연소득,근로형태,입사연월,고용형태,주거소유형태,대출희망금액,대출목적,개인회생자여부,개인회생자납입완료여부,기대출수,기대출금액
0,566528,681184,1974,남자,2022-03-01 00:11:35,580.0,8000000.0,OTHERINCOME,기입안함,기타,전월세,1000000.0,생활비,누락,누락,4.0,20000000.0
1,180433,623737,1997,남자,2022-03-01 00:12:05,740.0,12000000.0,FREELANCER,202202,일용직,기타가족소유,3000000.0,생활비,누락,누락,3.0,11000000.0
2,1657888,752985,1997,남자,2022-03-01 00:12:06,580.0,12000000.0,FREELANCER,202102,기타,기타가족소유,3000000.0,생활비,누락,누락,7.0,33000000.0


,신청서번호,유저번호,유저생년월일,유저성별,생성일시,한도조회당시유저신용점수,연소득,근로형태,입사연월,고용형태,주거소유형태,대출희망금액,대출목적,개인회생자여부,개인회생자납입완료여부,기대출수,기대출금액
0,501277,542827,1980,남자,2022-06-01 00:00:57,660.0,65000000.0,EARNEDINCOME,202001,계약직,자가,65000000.0,대환대출,비개인회생자,누락,5.0,151000000.0
1,731545,8641,1979,남자,2022-06-01 00:12:08,610.0,51000000.0,PRACTITIONER,201601,정규직,전월세,3000000.0,생활비,비개인회생자,누락,7.0,60000000.0
2,1343197,710829,1993,남자,2022-06-01 00:12:22,600.0,30000000.0,OTHERINCOME,기입안함,기타,자가,5000000.0,생활비,비개인회생자,누락,8.0,66000000.0


In [56]:
#입사연월 날짜형식통일
user_train['입사연월'] = user_train['입사연월'].apply(lambda x : x + '01' if (len(x)<8) and (len(x)>4)
                                     else x)
user_test['입사연월'] = user_test['입사연월'].apply(lambda x : x + '01' if (len(x)<8) and (len(x)>4)
                                     else x)

### -Loan result data

In [58]:
#결측치 확인
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
#            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return "완료"

display(check_missing_col(loan))

결측치가 있는 컬럼은: 승인한도입니다
해당 컬럼에 총 7495개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 승인금리입니다
해당 컬럼에 총 7495개의 결측치가 존재합니다.
결측치가 있는 컬럼은: 신청여부입니다
해당 컬럼에 총 3257239개의 결측치가 존재합니다.


'완료'

### Loan_result
 - application_id(신청서번호) = 중복값 존재, 결측치 x
 - loanapply_insert_time(한도조회일시) = 결측치 x
 - bank_id(금융사번호) = 63개의 고유값 존재, 결측치 x
 - product_id(상품번호) = 188개의 고유값 존재, 결측치 x
 - <span style="color:red">loan_limit(승인한도) = 7495개의 결측치가 존재</span>
 - <span style="color:red">loan_rate(승인금리) =  7495개의 결측치가 존재</span>

 - <span style="color:blue">is_applied(신청여부) = 560449개의 1, 9709675개의 0 그리고 나머지 예측값</span>

In [59]:
#한도조회일시 기준 3,4,5월 데이터는 train 6월 데이터는 test로 데이터 split(data leakage 방지)
loan['한도조회일시'] = loan['한도조회일시'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
loan = loan.sort_values(by="한도조회일시")

loan_train = loan[(loan['한도조회일시'].dt.month)<6].reset_index(drop=True)
loan_test = loan[(loan['한도조회일시'].dt.month)==6].reset_index(drop=True)

display(loan_train.head(3))
display(loan_test.head(3))

,신청서번호,한도조회일시,금융사번호,상품번호,승인한도,승인금리,신청여부
0,566528,2022-03-01 00:11:36,13,123,20000000.0,19.1,0.0
1,180433,2022-03-01 00:12:05,13,262,22000000.0,16.6,0.0
2,180433,2022-03-01 00:12:05,19,231,16000000.0,15.0,0.0


,신청서번호,한도조회일시,금융사번호,상품번호,승인한도,승인금리,신청여부
0,506292,2022-06-01 00:12:24,27,148,13000000.0,7.2,NaN
1,506292,2022-06-01 00:12:24,59,251,11000000.0,6.8,NaN
2,1255231,2022-06-01 00:12:24,29,265,19000000.0,11.4,NaN


In [60]:
data_train = pd.merge(loan_train.drop(columns=['신청여부']), user_train, how='left', on='신청서번호')
data_train = pd.concat([data_train,loan_train['신청여부']],axis=1)

data_test = pd.merge(loan_test.drop(columns=['신청여부']), user_test, how='left', on='신청서번호')
data_test = pd.concat([data_test,loan_test['신청여부']],axis=1)

display(data_train.head(3))
display(data_test.head(3))

,신청서번호,한도조회일시,금융사번호,상품번호,승인한도,승인금리,유저번호,유저생년월일,유저성별,생성일시,...,입사연월,고용형태,주거소유형태,대출희망금액,대출목적,개인회생자여부,개인회생자납입완료여부,기대출수,기대출금액,신청여부
0,566528,2022-03-01 00:11:36,13,123,20000000.0,19.1,681184.0,1974,남자,2022-03-01 00:11:35,...,기입안함,기타,전월세,1000000.0,생활비,누락,누락,4.0,20000000.0,0.0
1,180433,2022-03-01 00:12:05,13,262,22000000.0,16.6,623737.0,1997,남자,2022-03-01 00:12:05,...,20220201,일용직,기타가족소유,3000000.0,생활비,누락,누락,3.0,11000000.0,0.0
2,180433,2022-03-01 00:12:05,19,231,16000000.0,15.0,623737.0,1997,남자,2022-03-01 00:12:05,...,20220201,일용직,기타가족소유,3000000.0,생활비,누락,누락,3.0,11000000.0,0.0


,신청서번호,한도조회일시,금융사번호,상품번호,승인한도,승인금리,유저번호,유저생년월일,유저성별,생성일시,...,입사연월,고용형태,주거소유형태,대출희망금액,대출목적,개인회생자여부,개인회생자납입완료여부,기대출수,기대출금액,신청여부
0,506292,2022-06-01 00:12:24,27,148,13000000.0,7.2,669202.0,1981,남자,2022-06-01 00:12:23,...,20220101,정규직,기타가족소유,16000000.0,생활비,비개인회생자,누락,1.0,143000000.0,NaN
1,506292,2022-06-01 00:12:24,59,251,11000000.0,6.8,669202.0,1981,남자,2022-06-01 00:12:23,...,20220101,정규직,기타가족소유,16000000.0,생활비,비개인회생자,누락,1.0,143000000.0,NaN
2,1255231,2022-06-01 00:12:24,29,265,19000000.0,11.4,694012.0,1963,여자,2022-06-01 00:12:23,...,기입안함,기타,전월세,10000000.0,생활비,개인회생자,납입완료,6.0,43000000.0,NaN


In [61]:
data_train['한도조회월일'] = data_train['한도조회일시'].apply(lambda x : datetime.strptime(str(x)[:10], '%Y-%m-%d'))
data_test['한도조회월일'] = data_test['한도조회일시'].apply(lambda x : datetime.strptime(str(x)[:10], '%Y-%m-%d'))

In [62]:
data_train['한도조회월'] = data_train['한도조회일시'].dt.month
data_test['한도조회월'] = data_test['한도조회일시'].dt.month

In [63]:
data_train['상품코드'] = data_train['금융사번호'].astype(str) + '-' + data_train['상품번호'].astype(str)
data_test['상품코드'] = data_test['금융사번호'].astype(str) + '-' + data_test['상품번호'].astype(str)

In [64]:
#신청서번호기준 user_spec에는 없는데 loan_result에는 있는 데이터(반대도 있음 > 모델링 후 군집분석에 사용해야 될듯)
#user_spec 열의 값들이 결측치로 존재
#기간 차이로 인해 누락된 데이터 113개
#예측모델 학습에는 제외하되 군집분석에 사용할 여지가 있으므로 따로 변수에 저장
missing_train = data_train[data_train['유저번호'].isna()].reset_index(drop=True)
display(missing_train)

display(data_train.shape)
drop_index = data_train[data_train['유저번호'].isna()].index
data_train.drop(index=drop_index,inplace=True)
data_train = data_train.reset_index(drop=True)
display(data_train.shape)
display(data_train)

missing_test = data_test[data_test['유저번호'].isna()].reset_index(drop=True)
display(missing_test)

display(data_test.shape)
drop_index = data_test[data_test['유저번호'].isna()].index
data_test.drop(index=drop_index,inplace=True)
data_test = data_test.reset_index(drop=True)
display(data_test.shape)
display(data_test)

,신청서번호,한도조회일시,금융사번호,상품번호,승인한도,승인금리,유저번호,유저생년월일,유저성별,생성일시,...,대출희망금액,대출목적,개인회생자여부,개인회생자납입완료여부,기대출수,기대출금액,신청여부,한도조회월일,한도조회월,상품코드
0,1382876,2022-03-01 18:02:54,14,197,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2022-03-01,3,14-197
1,1669381,2022-03-01 18:03:44,14,197,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2022-03-01,3,14-197
2,1452711,2022-03-02 09:06:08,30,121,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2022-03-02,3,30-121
3,703783,2022-03-02 09:12:04,30,85,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2022-03-02,3,30-85
4,771476,2022-03-02 09:19:09,13,262,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2022-03-02,3,13-262
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108,450452,2022-05-20 17:53:11,22,124,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2022-05-20,5,22-124
109,1907865,2022-05-20 17:53:11,22,124,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2022-05-20,5,22-124
110,889541,2022-05-20 17:53:12,22,124,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2022-05-20,5,22-124
111,1979792,2022-05-20 17:53:59,22,124,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2022-05-20,5,22-124


(10270124, 26)

(10270011, 26)

,신청서번호,한도조회일시,금융사번호,상품번호,승인한도,승인금리,유저번호,유저생년월일,유저성별,생성일시,...,대출희망금액,대출목적,개인회생자여부,개인회생자납입완료여부,기대출수,기대출금액,신청여부,한도조회월일,한도조회월,상품코드
0,566528,2022-03-01 00:11:36,13,123,20000000.0,19.1,681184.0,1974,남자,2022-03-01 00:11:35,...,1000000.0,생활비,누락,누락,4.0,20000000.0,0.0,2022-03-01,3,13-123
1,180433,2022-03-01 00:12:05,13,262,22000000.0,16.6,623737.0,1997,남자,2022-03-01 00:12:05,...,3000000.0,생활비,누락,누락,3.0,11000000.0,0.0,2022-03-01,3,13-262
2,180433,2022-03-01 00:12:05,19,231,16000000.0,15.0,623737.0,1997,남자,2022-03-01 00:12:05,...,3000000.0,생활비,누락,누락,3.0,11000000.0,0.0,2022-03-01,3,19-231
3,1657888,2022-03-01 00:12:06,19,231,40000000.0,18.9,752985.0,1997,남자,2022-03-01 00:12:06,...,3000000.0,생활비,누락,누락,7.0,33000000.0,0.0,2022-03-01,3,19-231
4,1657888,2022-03-01 00:12:06,24,263,5000000.0,15.9,752985.0,1997,남자,2022-03-01 00:12:06,...,3000000.0,생활비,누락,누락,7.0,33000000.0,0.0,2022-03-01,3,24-263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10270006,455157,2022-05-31 23:54:28,35,168,3000000.0,18.3,717233.0,1973,남자,2022-05-31 23:54:27,...,50000000.0,대환대출,비개인회생자,납입중,12.0,293000000.0,0.0,2022-05-31,5,35-168
10270007,455157,2022-05-31 23:54:28,55,159,3000000.0,18.9,717233.0,1973,남자,2022-05-31 23:54:27,...,50000000.0,대환대출,비개인회생자,납입중,12.0,293000000.0,0.0,2022-05-31,5,55-159
10270008,455157,2022-05-31 23:54:29,22,221,5000000.0,18.4,717233.0,1973,남자,2022-05-31 23:54:27,...,50000000.0,대환대출,비개인회생자,납입중,12.0,293000000.0,0.0,2022-05-31,5,22-221
10270009,455157,2022-05-31 23:54:37,38,16,3000000.0,14.5,717233.0,1973,남자,2022-05-31 23:54:27,...,50000000.0,대환대출,비개인회생자,납입중,12.0,293000000.0,0.0,2022-05-31,5,38-16


,신청서번호,한도조회일시,금융사번호,상품번호,승인한도,승인금리,유저번호,유저생년월일,유저성별,생성일시,...,대출희망금액,대출목적,개인회생자여부,개인회생자납입완료여부,기대출수,기대출금액,신청여부,한도조회월일,한도조회월,상품코드
0,1753116,2022-06-01 21:04:31,14,197,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-01,6,14-197
1,354520,2022-06-02 09:02:35,30,85,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-02,6,30-85
2,235068,2022-06-02 10:04:46,30,85,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-02,6,30-85
3,1935529,2022-06-02 10:55:59,30,85,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-02,6,30-85
4,1386195,2022-06-02 11:35:31,51,21,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-02,6,51-21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,99676,2022-06-08 16:03:59,27,193,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-08,6,27-193
76,957092,2022-06-08 16:04:00,27,193,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-08,6,27-193
77,1130701,2022-06-08 16:04:03,27,193,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-08,6,27-193
78,1129641,2022-06-08 16:04:05,27,193,NaN,NaN,NaN,NaN,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-06-08,6,27-193


(3257239, 26)

(3257159, 26)

,신청서번호,한도조회일시,금융사번호,상품번호,승인한도,승인금리,유저번호,유저생년월일,유저성별,생성일시,...,대출희망금액,대출목적,개인회생자여부,개인회생자납입완료여부,기대출수,기대출금액,신청여부,한도조회월일,한도조회월,상품코드
0,506292,2022-06-01 00:12:24,27,148,13000000.0,7.2,669202.0,1981,남자,2022-06-01 00:12:23,...,16000000.0,생활비,비개인회생자,누락,1.0,143000000.0,NaN,2022-06-01,6,27-148
1,506292,2022-06-01 00:12:24,59,251,11000000.0,6.8,669202.0,1981,남자,2022-06-01 00:12:23,...,16000000.0,생활비,비개인회생자,누락,1.0,143000000.0,NaN,2022-06-01,6,59-251
2,1255231,2022-06-01 00:12:24,29,265,19000000.0,11.4,694012.0,1963,여자,2022-06-01 00:12:23,...,10000000.0,생활비,개인회생자,납입완료,6.0,43000000.0,NaN,2022-06-01,6,29-265
3,506292,2022-06-01 00:12:24,19,231,24000000.0,15.6,669202.0,1981,남자,2022-06-01 00:12:23,...,16000000.0,생활비,비개인회생자,누락,1.0,143000000.0,NaN,2022-06-01,6,19-231
4,506292,2022-06-01 00:12:25,56,5,2000000.0,18.5,669202.0,1981,남자,2022-06-01 00:12:23,...,16000000.0,생활비,비개인회생자,누락,1.0,143000000.0,NaN,2022-06-01,6,56-5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3257154,634459,2022-06-30 23:54:48,10,65,30000000.0,14.5,244440.0,2000,남자,2022-06-30 23:54:47,...,5000000.0,생활비,비개인회생자,납입중,1.0,127000000.0,NaN,2022-06-30,6,10-65
3257155,634459,2022-06-30 23:54:48,10,149,30000000.0,14.5,244440.0,2000,남자,2022-06-30 23:54:47,...,5000000.0,생활비,비개인회생자,납입중,1.0,127000000.0,NaN,2022-06-30,6,10-149
3257156,634459,2022-06-30 23:54:48,19,231,11000000.0,16.6,244440.0,2000,남자,2022-06-30 23:54:47,...,5000000.0,생활비,비개인회생자,납입중,1.0,127000000.0,NaN,2022-06-30,6,19-231
3257157,1288711,2022-06-30 23:54:52,35,267,3000000.0,13.8,450880.0,1976,여자,2022-06-30 23:54:44,...,50000000.0,대환대출,비개인회생자,누락,4.0,96000000.0,NaN,2022-06-30,6,35-267


In [65]:
#승인금리 이상치 처리(없음)

#승인금리 결측치 대체(상품코드별 승인금리값을 기준으로 채움)
data_train['승인금리'] = data_train['승인금리'].fillna(data_train.groupby('상품코드')['승인금리'].transform('mean'))
data_train['승인금리'] = data_train['승인금리'].fillna(data_train['승인금리'].mean())

data_test['승인금리'] = data_test['승인금리'].fillna(data_train.groupby('상품코드')['승인금리'].transform('mean'))
data_test['승인금리'] = data_test['승인금리'].fillna(data_train['승인금리'].mean())

#데이터 통일성을 위한 처리
data_train['승인금리'] = data_train['승인금리'].apply(lambda x : round(x,2))
data_test['승인금리'] = data_test['승인금리'].apply(lambda x : round(x,2))

In [66]:
#승인한도 이상치 처리(승인한도가 0인 경우) > 결측치가 아니고 0인 이유가 있다고 판단하여 보존함

#승인한도 결측치 대체(상품코드별 승인한도값을 기준으로 채움)
data_train['승인한도'] = data_train['승인한도'].fillna(data_train.groupby('상품코드')['승인한도'].transform('mean'))
data_train['승인한도'] = data_train['승인한도'].fillna(data_train['승인한도'].mean())

data_test['승인한도'] = data_test['승인한도'].fillna(data_train.groupby('상품코드')['승인한도'].transform('mean'))
data_test['승인한도'] = data_test['승인한도'].fillna(data_train['승인한도'].mean())

#데이터 통일성을 위한 처리
data_train['승인한도'] = data_train['승인한도'].apply(lambda x : round(x,-5))
data_test['승인한도'] = data_test['승인한도'].apply(lambda x : round(x,-5))

In [74]:
data_train.columns

Index(['신청서번호', '한도조회일시', '금융사번호', '상품번호', '승인한도', '승인금리', '유저번호', '유저생년월일',
       '유저성별', '생성일시', '한도조회당시유저신용점수', '연소득', '근로형태', '입사연월', '고용형태', '주거소유형태',
       '대출희망금액', '대출목적', '개인회생자여부', '개인회생자납입완료여부', '기대출수', '기대출금액', '신청여부',
       '한도조회월일', '한도조회월', '상품코드'],
      dtype='object')

In [ ]:
'유저타입', '상품매력도', '연이자부담지수', '3-5월평균순위변동폭',
       '6월예상순위', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '연소득증감률',
       '과대출자', '기출등급', '나이', '신청서조회_비율', '신용점수_증감률'

## 3. Feature Engineering

#### 유저타입

In [76]:
#유저타입 feature 생성
frequency = data_train.groupby('유저번호')['신청서번호'].agg([('대출조회횟수','nunique')])
applied_count = data_train.groupby('유저번호')['신청여부'].agg([('신청횟수','sum')])

data = {
    '대출조회횟수' : np.sort(frequency.대출조회횟수.unique()),
    'Count' : 0,
    'Ratio' : 0,
    '누적 값' : 0
}
index_name = np.sort(frequency.대출조회횟수.unique())

frequency_data = pd.DataFrame(data, index = index_name)
frequency_data['Count'] = frequency.대출조회횟수.value_counts().sort_index(ascending=True)
frequency_data['Ratio'] = round(frequency_data['Count']/259328,3)
frequency_data['누적 값'] = frequency_data['Ratio'].cumsum()
display(frequency_data)
#평균 대출조회횟수 = 2.8
#대출조회횟수가 3인 유저까지 소극적 이용자로 정의
feature_frame1 = pd.concat([frequency,applied_count],axis=1)
feature_frame1['유저타입'] = feature_frame1['대출조회횟수'].apply(lambda x : "passive" if (x<=3)
                                                            else "active")

#vip_level(핵심고객) feature 생성
#feature_frame1['vip_level'] = feature_frame1['신청횟수']/feature_frame1['대출조회횟수']

feature_frame1 = feature_frame1.drop(columns=['대출조회횟수','신청횟수'])
feature_frame1['유저번호'] = feature_frame1.index
feature_frame1 = feature_frame1.reset_index(drop=True)

data_train = pd.merge(data_train, feature_frame1, how='left', on='유저번호')
data_test = pd.merge(data_test, feature_frame1, how='left', on='유저번호')

#user_type 결측치 대체
data_test['유저타입'] = data_test['유저타입'].fillna("passive")

#vip_level 결측치 대체
#data_test['vip_level'] = data_test['vip_level'].fillna(data_train['vip_level'].mean())

,대출조회횟수,Count,Ratio,누적 값
1,1,128538,0.496,0.496
2,2,50189,0.194,0.690
3,3,26138,0.101,0.791
4,4,15968,0.062,0.853
5,5,9939,0.038,0.891
...,...,...,...,...
145,145,1,0.000,0.998
150,150,1,0.000,0.998
160,160,1,0.000,0.998
161,161,1,0.000,0.998


#### 상품매력도

In [77]:
#상품매력도 feature 생성
data_train['pd_at1'] = data_train['승인한도'] - data_train['대출희망금액']

def diff_func(x):
    if x>=0 :
        return 1
    else :
        if x <= -2.700000e+07:
            return -1 * 1.5 # 25%
        elif x <= -6.000000e+06:
            return -1 * 1.0 # 50%
        else :
            return -1 * 0.5
data_train['pd_at1'] = data_train['pd_at1'].apply(diff_func)
data_train['상품매력도'] = data_train['승인금리'] * data_train['pd_at1']
data_train = data_train.drop(columns=['pd_at1'])

#### 연이자부담지수

In [78]:
#연이자부담지수 feature 생성
data_train['연이자부담지수'] = data_train['승인금리'] * 0.01 * data_train['대출희망금액'] 
data_train['연이자부담지수'] = data_train['연이자부담지수']/data_train['연소득']

In [79]:
data_train[data_train['연이자부담지수']==np.inf].근로형태.value_counts()

OTHERINCOME        20080
EARNEDINCOME        2788
PRIVATEBUSINESS     2229
FREELANCER          2117
EARNEDINCOME2        438
PRACTITIONER          90
Name: 근로형태, dtype: int64

In [80]:
#연소득이 0인경우 inf값 처리
#근로형태가 OTHERINCOME인 경우 주식이나 코인과 같은 비노동적 소득이라고 판단됨 > 연이자부담지수의 25%로 대체
#이외의 근로형태의 경우 > 연이자부담지수의 75%로 대체
data_train[data_train['근로형태']=='OTHERINCOME']['연이자부담지수'] = data_train[data_train['근로형태']=='OTHERINCOME']['연이자부담지수'].replace(np.inf,3.850000e-02) #25%
data_train['연이자부담지수'] = data_train['연이자부담지수'].replace(np.inf,1.486111e-01) #75%

In [81]:
data_test['pd_at1'] = data_test['승인한도'] - data_test['대출희망금액']
data_test['pd_at1'] = data_test['pd_at1'].apply(diff_func)
data_test['상품매력도'] = data_test['승인금리'] * data_test['pd_at1']
data_test = data_test.drop(columns=['pd_at1'])

data_test['연이자부담지수'] = data_test['승인금리'] * 0.01 * data_test['대출희망금액']
data_test['연이자부담지수'] = data_test['연이자부담지수']/data_test['연소득']

In [82]:
data_train['연이자부담지수'] = data_train['연이자부담지수'].fillna(0)
data_test['연이자부담지수'] = data_test['연이자부담지수'].fillna(0)

In [83]:
data_test[data_test['근로형태']=='OTHERINCOME']['연이자부담지수'] = data_test[data_test['근로형태']=='OTHERINCOME']['연이자부담지수'].replace(np.inf,3.850000e-02) #25%
data_test['연이자부담지수'] = data_test['연이자부담지수'].replace(np.inf,1.486111e-01) #75%

In [84]:
gc.collect()

172

#### 3-5월 평균순위변동폭 / 6월예상순위

In [85]:
#3-5월평균순위변동폭 및 6월예상순위 피처생성
data_train.groupby(['상품코드','한도조회월'])['유저번호'].agg([('월별추천건수','nunique')])

월별추천건수
상품코드  한도조회월        
1-1   3       32762
      4        9392
1-61  3       27546
      4       48153
      5       52980
...             ...
8-31  4       16234
      5       18869
9-105 5        2249
9-190 5           4
9-250 5        1043

[477 rows x 1 columns]

In [86]:
march = data_train[data_train['한도조회월']==3].groupby(['상품코드'])['유저번호'].agg([('3월추천건수','nunique')]).sort_values(by='3월추천건수',ascending=False)
march['3월순위'] = march['3월추천건수'].rank(method='average',ascending=False)
march['상품코드'] = march.index
march.reset_index(drop=True, inplace=True)
display(march.sort_values(by='3월순위'))

rank_data = {
    '상품코드' : data_train.상품코드.unique()
}

rank_data = pd.DataFrame(rank_data)
rank_data = pd.merge(rank_data, march, how='left',on='상품코드')
display(rank_data)

,3월추천건수,3월순위,상품코드
0,46456,1.0,6-36
1,43216,2.0,59-150
2,39410,3.0,32-56
3,36457,4.0,50-142
4,33936,5.0,35-29
...,...,...,...
148,3,149.5,35-146
149,3,149.5,27-193
150,1,152.0,27-240
151,1,152.0,51-46


,상품코드,3월추천건수,3월순위
0,13-123,18475.0,42.0
1,13-262,30928.0,13.0
2,19-231,16576.0,49.0
3,24-263,5165.0,107.0
4,14-197,17347.0,46.0
...,...,...,...
173,47-181,NaN,NaN
174,36-55,NaN,NaN
175,9-250,NaN,NaN
176,9-190,NaN,NaN


In [87]:
april = data_train[data_train['한도조회월']==4].groupby(['상품코드'])['유저번호'].agg([('4월추천건수','nunique')]).sort_values(by='4월추천건수',ascending=False)
april['4월순위'] = april['4월추천건수'].rank(method='average',ascending=False)
april['상품코드'] = april.index
april.reset_index(drop=True, inplace=True)
display(april.sort_values(by='4월순위'))

rank_data = pd.merge(rank_data, april, how='left',on='상품코드')
display(rank_data)

,4월추천건수,4월순위,상품코드
0,48153,1.0,1-61
1,46235,2.0,6-36
2,41741,3.0,59-150
3,38845,4.0,50-142
4,36589,5.0,49-39
...,...,...,...
156,1,159.0,34-52
157,1,159.0,49-225
158,1,159.0,51-46
159,1,159.0,2-165


,상품코드,3월추천건수,3월순위,4월추천건수,4월순위
0,13-123,18475.0,42.0,18923.0,44.0
1,13-262,30928.0,13.0,31446.0,13.0
2,19-231,16576.0,49.0,16554.0,55.0
3,24-263,5165.0,107.0,NaN,NaN
4,14-197,17347.0,46.0,17351.0,50.0
...,...,...,...,...,...
173,47-181,NaN,NaN,NaN,NaN
174,36-55,NaN,NaN,NaN,NaN
175,9-250,NaN,NaN,NaN,NaN
176,9-190,NaN,NaN,NaN,NaN


In [88]:
may = data_train[data_train['한도조회월']==5].groupby(['상품코드'])['유저번호'].agg([('5월추천건수','nunique')]).sort_values(by='5월추천건수',ascending=False)
may['5월순위'] = may['5월추천건수'].rank(method='average',ascending=False)
may['상품코드'] = may.index
may.reset_index(drop=True, inplace=True)
display(may.sort_values(by='5월순위'))

rank_data = pd.merge(rank_data, may, how='left',on='상품코드')
display(rank_data)

,5월추천건수,5월순위,상품코드
0,52980,1.0,1-61
1,45514,2.0,35-29
2,39392,3.0,33-110
3,37938,4.0,50-142
4,37561,5.0,49-39
...,...,...,...
158,4,158.5,9-190
159,3,160.0,27-240
160,2,161.5,36-55
161,2,161.5,2-165


,상품코드,3월추천건수,3월순위,4월추천건수,4월순위,5월추천건수,5월순위
0,13-123,18475.0,42.0,18923.0,44.0,20760.0,39.0
1,13-262,30928.0,13.0,31446.0,13.0,33170.0,10.0
2,19-231,16576.0,49.0,16554.0,55.0,18103.0,51.0
3,24-263,5165.0,107.0,NaN,NaN,249.0,140.0
4,14-197,17347.0,46.0,17351.0,50.0,18258.0,50.0
...,...,...,...,...,...,...,...
173,47-181,NaN,NaN,NaN,NaN,3384.0,120.0
174,36-55,NaN,NaN,NaN,NaN,2.0,161.5
175,9-250,NaN,NaN,NaN,NaN,1043.0,133.0
176,9-190,NaN,NaN,NaN,NaN,4.0,158.5


In [89]:
rank_data.drop(columns=['3월추천건수','4월추천건수','5월추천건수'],inplace=True)
rank_data

,상품코드,3월순위,4월순위,5월순위
0,13-123,42.0,44.0,39.0
1,13-262,13.0,13.0,10.0
2,19-231,49.0,55.0,51.0
3,24-263,107.0,NaN,140.0
4,14-197,46.0,50.0,50.0
...,...,...,...,...
173,47-181,NaN,NaN,120.0
174,36-55,NaN,NaN,161.5
175,9-250,NaN,NaN,133.0
176,9-190,NaN,NaN,158.5


In [90]:
rank_data['3월순위'] = rank_data['3월순위'].fillna(rank_data['3월순위'].max())
rank_data['4월순위'] = rank_data['4월순위'].fillna(rank_data['4월순위'].max())
rank_data['5월순위'] = rank_data['5월순위'].fillna(rank_data['5월순위'].max())

In [92]:
rank_data['3-4월변동순위'] = rank_data['3월순위'] - rank_data['4월순위']
rank_data['4-5월변동순위'] = rank_data['4월순위'] - rank_data['5월순위']
rank_data['3-5월평균순위변동폭'] = (rank_data['3-4월변동순위'] + rank_data['4-5월변동순위'])/2

rank_data['6월예상순위'] = rank_data['5월순위'] + rank_data['3-5월평균순위변동폭']

In [93]:
rank_data = rank_data[['상품코드','3-5월평균순위변동폭','6월예상순위']]
rank_data.head()

,상품코드,3-5월평균순위변동폭,6월예상순위
0,13-123,1.5,40.5
1,13-262,1.5,11.5
2,19-231,-1.0,50.0
3,24-263,-16.5,123.5
4,14-197,-2.0,48.0


In [94]:
data_train = pd.merge(data_train, rank_data, how='left',on='상품코드')
data_test = pd.merge(data_test, rank_data, how='left',on='상품코드')

data_test['3-5월평균순위변동폭'] = data_test['3-5월평균순위변동폭'].fillna(round(data_train['3-5월평균순위변동폭'].mean(),2))
data_test['6월예상순위'] = data_test['6월예상순위'].fillna(round(data_train['6월예상순위'].mean(),2))

In [95]:
gc.collect()

15

#### 연소득 증감률

In [96]:
tr_min = data_train.groupby('유저번호')['연소득'].min().reset_index().rename(columns = {'연소득' : '연소득증감률'})
tr_max = data_train.groupby('유저번호')['연소득'].max().reset_index().rename(columns = {'연소득' : '연소득증감률'})

a = tr_min.유저번호.reset_index()
b = np.log1p((tr_max['연소득증감률']-tr_min['연소득증감률'])/(tr_min['연소득증감률']+(10e-16))*100)
tr_연소득증감률 = pd.concat([a, b], axis = 1).iloc[:,1:]

In [97]:
te_min = data_test.groupby('유저번호')['연소득'].min().reset_index().rename(columns = {'연소득' : '연소득증감률'})
te_max = data_test.groupby('유저번호')['연소득'].max().reset_index().rename(columns = {'연소득' : '연소득증감률'})

a = te_min.유저번호.reset_index()
b = np.log1p((te_max['연소득증감률']-te_min['연소득증감률'])/(te_min['연소득증감률']+(10e-16))*100)
te_연소득증감률 = pd.concat([a, b], axis = 1).iloc[:,1:]

In [98]:
data_train = pd.merge(data_train, tr_연소득증감률, how = 'left', on = '유저번호')
data_test = pd.merge(data_test, te_연소득증감률, how = 'left', on = '유저번호')

#### 과대출자

In [99]:
# 연소득과 대출희망금액의 차이와 신청여부간의 관계 파악
# 기대출금액 + 대출희망금액 - 연소득 < 0  -> 연소득보다 대출금액이 더 많은 과대출자로 분류

In [100]:
data_train['과대출자'] = np.nan
data_train.loc[(data_train['연소득'] + data_train['기대출금액']) < data_train['대출희망금액'], '과대출자'] = 'N'
data_train.loc[(data_train['연소득'] + data_train['기대출금액']) >= data_train['대출희망금액'], '과대출자'] = 'Y'

data_test['과대출자'] = np.nan
data_test.loc[(data_test['연소득'] + data_test['기대출금액']) < data_test['대출희망금액'], '과대출자'] = 'N'
data_test.loc[(data_test['연소득'] + data_test['기대출금액']) >= data_test['대출희망금액'], '과대출자'] = 'Y'

#### 기출등급

기대출수와 기대출금액을 총 5개의 구간으로 나눈 후, 둘의 합을 다시 5개의 구간으로

In [101]:
data_train['기대출수_등급'] = data_train['기대출수'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.2, 0.4, 0.6, 0.8, 1], labels = range(1,6))).astype(int)

data_test['기대출수_등급'] = data_test['기대출수'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.2, 0.4, 0.6, 0.8, 1], labels = range(1,6))).astype(int)

In [102]:
data_train['기대출금액_등급'] = data_train['기대출금액'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.2, 0.4, 0.6, 0.8, 1], labels = range(1,6))).astype(int)

data_test['기대출금액_등급'] = data_test['기대출금액'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.2, 0.4, 0.6, 0.8, 1], labels = range(1,6))).astype(int)

In [103]:
data_train['기출등급'] = data_train['기대출수_등급'] + data_train['기대출금액_등급']
data_train['기출등급'] = data_train['기출등급'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.2, 0.4, 0.6, 0.8, 1], labels = range(1,6))).astype(int)

data_test['기출등급'] = data_test['기대출수_등급'] + data_test['기대출금액_등급']
data_test['기출등급'] = data_test['기출등급'].transform(lambda x: pd.qcut(x.rank(method='first'), q = [0, 0.2, 0.4, 0.6, 0.8, 1], labels = range(1,6))).astype(int)

In [104]:
del data_train['기대출수_등급']
del data_train['기대출금액_등급']

del data_test['기대출수_등급']
del data_test['기대출금액_등급']

#### 사회생활

In [105]:
data_train['유저생년월일'] = data_train.유저생년월일.replace('기입안함',2023)
data_train['유저생년월일'] = data_train['유저생년월일'].astype(int)

data_test['유저생년월일'] = data_test.유저생년월일.replace('기입안함',2023)
data_test['유저생년월일'] = data_test['유저생년월일'].astype(int)

In [106]:
data_train['나이'] = 2023 - data_train['유저생년월일']

data_test['나이'] = 2023 - data_test['유저생년월일']

In [107]:
data_train['나이'] = data_train['나이'].apply(lambda x: '10대' if 10 <= x < 20
                        else '20대' if 20 <= x < 30 
                        else '30대' if 30 <= x < 40
                        else '40대' if 40 <= x < 50
                        else '50대' if 50 <= x < 60
                        else '60대' if 60 <= x < 70
                        else '70대' if 70 <= x < 80
                        else '80대' if 80 <= x < 90
                        else '90대' if 90 <= x < 100
                        else '기입안함')


data_test['나이'] = data_test['나이'].apply(lambda x: '10대' if 10 <= x < 20
                        else '20대' if 20 <= x < 30 
                        else '30대' if 30 <= x < 40
                        else '40대' if 40 <= x < 50
                        else '50대' if 50 <= x < 60
                        else '60대' if 60 <= x < 70
                        else '70대' if 70 <= x < 80
                        else '80대' if 80 <= x < 90
                        else '90대' if 90 <= x < 100
                        else '기입안함')

In [108]:
data_train['나이'] = data_train['나이'].replace('10대','미성년자').replace('20대','사회초년생').replace('30대','사원/대리급').replace(
                    '40대','과장/차장급').replace('50대','부장급').replace('60대','퇴직대상').replace({'70대','80대','90대'},'사회노년층')


data_test['나이'] = data_test['나이'].replace('10대','미성년자').replace('20대','사회초년생').replace('30대','사원/대리급').replace(
                    '40대','과장/차장급').replace('50대','부장급').replace('60대','퇴직대상').replace({'70대','80대','90대'},'사회노년층')

In [109]:
gc.collect()

116

#### 신청서 조회 비율

In [111]:
a = data_train.groupby('유저번호')['신청서번호'].count().reset_index().rename(columns = {'신청서번호' : '신청서조회_비율'})
tr_신청서_조회 = a.신청서조회_비율/a.신청서조회_비율.sum()
tr_신청서_조회 = pd.concat([a['유저번호'],tr_신청서_조회], axis = 1)

In [112]:
a = data_test.groupby('유저번호')['신청서번호'].count().reset_index().rename(columns = {'신청서번호' : '신청서조회_비율'})
te_신청서_조회 = a.신청서조회_비율/a.신청서조회_비율.sum()
te_신청서_조회 = pd.concat([a['유저번호'],te_신청서_조회], axis = 1)

In [113]:
data_train = pd.merge(data_train, tr_신청서_조회, how = 'left', on = '유저번호')
data_test = pd.merge(data_test, te_신청서_조회, how = 'left', on = '유저번호')

#### 신용점수 증감률

In [114]:
tr_min = data_train.groupby('유저번호')['한도조회당시유저신용점수'].min().reset_index().rename(columns = {'한도조회당시유저신용점수' : '신용점수_증감률'})
tr_max = data_train.groupby('유저번호')['한도조회당시유저신용점수'].max().reset_index().rename(columns = {'한도조회당시유저신용점수' : '신용점수_증감률'})

a = tr_min.유저번호.reset_index()
b = np.log1p((tr_max['신용점수_증감률']-tr_min['신용점수_증감률'])/(tr_min['신용점수_증감률']+(10e-16))*100)
tr_신용점수_증감률 = pd.concat([a, b], axis = 1).iloc[:,1:]

In [115]:
te_min = data_test.groupby('유저번호')['한도조회당시유저신용점수'].min().reset_index().rename(columns = {'한도조회당시유저신용점수' : '신용점수_증감률'})
te_max = data_test.groupby('유저번호')['한도조회당시유저신용점수'].max().reset_index().rename(columns = {'한도조회당시유저신용점수' : '신용점수_증감률'})

a = te_min.유저번호.reset_index()
b = np.log1p((te_max['신용점수_증감률']-te_min['신용점수_증감률'])/(te_min['신용점수_증감률']+(10e-16))*100)
te_신용점수_증감률 = pd.concat([a, b], axis = 1).iloc[:,1:]

In [116]:
data_train = pd.merge(data_train, tr_신용점수_증감률, how = 'left', on = '유저번호')
data_test = pd.merge(data_test, te_신용점수_증감률, how = 'left', on = '유저번호')

In [118]:
data_train.shape, data_test.shape

((10270011, 37), (3257159, 37))

#### 앱버전

In [142]:
log = pd.read_csv('data/log_data.csv')

In [162]:
version_info = pd.DataFrame({'mp_app_version':['3.4.0','3.4.1','3.5.0','3.6.0','3.6.1', '3.7.0','3.8.0','3.10.0','3.10.1','3.10.2','3.12.0','3.12.1','3.13.0','3.13.1','3.14.0','3.15.0','3.15.1','3.16.0','3.17.0','3.18.0','3.19.0','3.20.0','3.21.0','3.22.0','3.23.0'],
              '앱버전_세부내역' : ['대출 신청 전에 대출신청 및 입금까지의 예상 시간 확인 가능, 고객 센터 답변을 편하게 받는 기능 추가',
                          '대출금을 값아주는 보험 추가, 대출 신청 전에 대출신청 및 입금까지의 예상 시간 확인 가능, 고객 센터 답변을 편하게 받는 기능 추가',
                          '공동인증서를 대신한 네이버/카카오/PASS 인증서를 간편인증서로 사용하여 정확하게 대출 확인 가능',
                          '장기렌트 서비스와 리스 서비스 출시로 리스렌트 견적서를 받아 5일 도착으로 차를 받을 수 있음',
                          '장기렌트 서비스와 리스 서비스 출시로 리스렌트 견적서를 받아 5일 도착으로 차를 받을 수 있음',
                          '자동차 구매 대출 출시로 현대/기아/제네시스를 저렴하게 이용 가능',
                          '개인회생자 전용 대출 출시',
                          '마이데이터로 내 대출 계좌를 연동하여 정확한 상환 일정 알림과 대환대출 여부 등 대출 관리 가능',
                          '카드 대출 누락 문제 수정',
                          '마이데이터로 내 대출 계좌를 연동하여 정확한 상환 일정 알림과 대환대출 여부 등 대출 관리 가능',
                          '마이데이터로 내 대출 계좌를 연동하여 정확한 상환 일정 알림과 대환대출 여부 등 대출 관리 가능, 여윳돈 계산기와 DSR 계산기 개선',
                          '몇몇 오류와 기능 개선',
                          '대출 실행까지 얼마나 걸리는지 알려주는 기능 출시',
                          '대출 실행까지 얼마나 걸리는지 알려주는 기능 출시',
                          '기존정보로 빠르게 나의 대출한도를 확인할 수 있는 등 대출조회를 간편하게 할 수 있음',
                          '마이데이터로 상환계좌를 연결하고 상환계좌의 잔액 확인 가능',
                          '마이데이터로 상환계좌를 연결하고 상환계좌의 잔액 확인 가능',
                          '신용 대출 뿐 아니라 아파트 후순위 담보 대출도 비교 가능',
                          '금융사 연결 과정을 개선해서 더 편리하게 내 계좌 연결 가능',
                          '금융사 연결 과정을 개선해서 더 편리하게 내 계좌 연결 가능',
                          '아파트 후순위 담보 대출 주소 검색을 위한 안내 추가',
                          '아파트 후순위 담보 대출 주소 검색을 위한 안내 추가',
                          '아파트 후순위 담보 대출 주소 검색을 위한 안내 추가',
                          '아파트 후순위 담보 대출 주소 검색을 위한 안내 추가',
                          '대출 관리 기능을 개선하여 대출 쉽게 관리 가능'
                          ]})

In [163]:
log_version = log[['user_id','mp_app_version', 'date_cd']].merge(version_info, on='mp_app_version', how='left')
log_version = log_version.fillna('정보 없음').rename(columns={'user_id':'유저번호','date_cd':'한도조회월일', 'mp_app_version':'앱버전'})

In [164]:
log_version = log_version.drop_duplicates(['유저번호','한도조회월일'])

In [165]:
log_version['한도조회월일'] = log_version.한도조회월일.astype('datetime64')

In [172]:
data_train = data_train.merge(log_version, on=['유저번호','한도조회월일'], how='left')
data_test = data_test.merge(log_version, on=['유저번호','한도조회월일'], how='left')

In [178]:
data_train = data_train.fillna('정보없음')

In [179]:
data_test['앱버전'] = data_test.앱버전.fillna('정보없음')
data_test['앱버전_세부내역'] = data_test.앱버전_세부내역.fillna('정보없음')

In [180]:
gc.collect()

60

#### Sequence in Log

In [ ]:
# 대용량의 데이터의 sequence를 뽑기 위해 별도의 python파일을 생성하여 터미널에서 작동.
# 파일은 별도 첨부하겠음.
# 코드는 아래와 같음.

__`Sequence_in_log.py`__

```python
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tqdm import tqdm
import pickle

def run_df():
    loan = pd.read_csv('loan_result.csv')
    log = pd.read_csv('log_data.csv')
    user = pd.read_csv('user_spec.csv')
    id_in_loan = np.load('id_in_loan.npy')
    id_in_loan = pd.Series(id_in_loan)
    id_in_loan = id_in_loan.rename('id_in_loan')
    user = pd.concat([user,id_in_loan],axis=1)
    return loan, log, user

def create_dt():
    log['timestamp'] = pd.to_datetime(log['timestamp'])
    user['insert_time'] = pd.to_datetime(user['insert_time'])
    loan['loanapply_insert_time'] = pd.to_datetime(loan['loanapply_insert_time'])
    user['month'] = user['insert_time'].dt.month
    log['month'] = log['timestamp'].dt.month
    log['day'] = log.timestamp.dt.day
    log['hour'] = pd.to_datetime(log['timestamp']).dt.hour
    
def user_seq(loan,log,user):
    user_list_log = log.user_id.unique()
    log = log.sort_values(by = 'timestamp')
    log_mini = log[['user_id','event','timestamp','month','day','hour']]
    user_sq = dict()
    idx_val = 0
    for i in tqdm(user_list_log):
        mini_data = log_mini.query('user_id == @i')
        time_list = []
        for j in range(mini_data.shape[0]):
            val = str(list(mini_data.iloc[j,-3 : ].values))
            time_list.append(val)
        # mini_data.query('month == @val[0]')
        time_list = set(time_list)
        user_list = [[] for i in range(len(time_list))]
        idx = 0
        tracker = []
        for z in range(mini_data.shape[0]):
            if z == 0 :
                user_list[idx].append([mini_data['event'].values[z],mini_data['timestamp'].values[z]])
                tracker.append(mini_data['hour'].values[z])
            else:
                if mini_data['hour'].values[z] != tracker[z-1]:
                    if (mini_data['timestamp'].values[z] - mini_data['timestamp'].values[z-1]).astype(int) > 300:
                        idx +=1
                        user_list[idx].append([mini_data['event'].values[z],mini_data['timestamp'].values[z]])
                        tracker.append(mini_data['hour'].values[z])
                    else:
                        user_list[idx].append([mini_data['event'].values[z],mini_data['timestamp'].values[z]])
                        tracker.append(mini_data['hour'].values[z])
                else:
                    user_list[idx].append([mini_data['event'].values[z],mini_data['timestamp'].values[z]])
                    tracker.append(mini_data['hour'].values[z])
        idx_val+=1
        user_sq[i] = user_list
        if idx_val <5:
            print('example :', user_sq[i])
        elif idx_val == 5:
            with open('user1.pickle','wb') as fw:
                pickle.dump(user_sq, fw)
            file = 'user1.pickle'
            with open(file, 'rb') as f:
                data = pickle.load(f)
            print('Pickle_RESULT',data)
        else:
            pass
                
    return user_sq

def add_app_id(user_sq):
    min_list = list(user_sq.keys())
    app_result = []
    for i in tqdm(min_list):
        mini_data = user.query('user_id == @i')[['application_id','label','insert_time']]
        for tracker,seq in enumerate(user_sq[i]): # 한 유저의 모든 시퀀스 
            # seq 는 한 유저 하나의 시퀀스를 의미한다.
            for idx in range(mini_data.shape[0]): # 해당 유저의 user 데이터
                if len(seq) ==0 :
                    pass
                elif abs(mini_data.iloc[idx,-1] - pd.to_datetime(seq[-1][-1])).days <= 1:
                    if abs(mini_data.iloc[idx,-1] - seq[-1][-1]).seconds <= 300:
                        user_sq[i][tracker].append([mini_data.iloc[idx,0],mini_data.iloc[idx,-1]])
                        pass
                else:
                    pass
    return user_sq

def there_in(x):
    loan_app_list = user.query('id_in_loan == 1').application_id.unique()
    if x in loan_app_list:
        if (loan.loc[(loan['application_id'] == x)]['is_applied'] == 1).sum() >=1:
            return 2
        elif (loan.loc[(loan['application_id'] == x)]['is_applied'] == 0).sum() == 0:
            return 1
        else:
            return 4
    else:
        return 0
if __name__ == "__main__":
    loan, log, user = run_df()
    create_dt()
    user_sq = user_seq(loan,log,user)
    with open('user.pickle','wb') as fw:
        pickle.dump(user_sq, fw)
    #user['label'] = user.application_id.apply(lambda x : there_in(x))
    print(user.columns)
    user_sq1 = add_app_id(user_sq)
    with open('user_f.pickle','wb') as fw:
        pickle.dump(user_sq1, fw)

```

In [186]:
# Sequence_in_log.py 를 통해 나온 sequence를 담고 있는 피쳐
sequence_loan = pd.read_csv('data/sequence_loan.csv')

In [188]:
sequence_loan = sequence_loan.rename(columns = {'application_id' : '신청서번호', 'loanapply_insert_time' : '한도조회일시'})

In [189]:
#한도조회일시 기준 3,4,5월 데이터는 train 6월 데이터는 test로 데이터 split(data leakage 방지)
sequence_loan['한도조회일시'] = sequence_loan['한도조회일시'].apply(lambda x : datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
sequence_loan = sequence_loan.sort_values(by="한도조회일시")

loan_train = sequence_loan[(sequence_loan['한도조회일시'].dt.month)<6].reset_index(drop=True)
loan_test = sequence_loan[(sequence_loan['한도조회일시'].dt.month)==6].reset_index(drop=True)

In [214]:
id_idx = data_train.신청서번호.unique().tolist()
id_idx2 = data_test.신청서번호.unique().tolist()

In [215]:
loan_train = loan_train.query('신청서번호 == @id_idx')
loan_test = loan_test.query('신청서번호 == @id_idx2')

In [216]:
loan_train = loan_train[['신청서번호','0','1','2','3','4','5','6','7','8','9']]
loan_test = loan_test[['신청서번호','0','1','2','3','4','5','6','7','8','9']]

In [218]:
loan_train = loan_train.sort_values(by = '신청서번호')
loan_test = loan_test.sort_values(by = '신청서번호')

In [219]:
data_train.shape, loan_train.shape

((10270011, 39), (10270011, 11))

In [220]:
for i in range(10):
    data_train[f'{i}'] =  loan_train[f'{i}'].values
    data_test[f'{i}'] =  loan_test[f'{i}'].values

In [217]:
gc.collect()

101

## to_csv

In [222]:
data_train.to_csv('data/data_train.csv', index = False)
data_test.to_csv('data/data_test.csv', index = False)